# <b> Text classification with tf.keras

In this notebook, we will classify reviews of IMDB dataset

<b> Importing packages</b>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0-dev20200514


In this notebook, we will be using tensorflow developer version 2.3.0-dev20200514 for our task in hand.

<b> Loading data </b>

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  11.3M      0  0:00:07  0:00:07 --:--:-- 17.9M


IMDB review sentiment dataset is an open source dataset from standford.

<b>Checking directory structure</b>

In [3]:
# !ls will list the directory contents 
!ls #checking the folders downloaded

aclImdb  aclImdb_v1.tar.gz  logs1  sample_data


In [4]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [6]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [7]:
import os
os.listdir('aclImdb/train/pos')[:5]

['1312_9.txt', '7551_10.txt', '12493_8.txt', '6284_7.txt', '3545_7.txt']

In [8]:
!cat aclImdb/train/pos/1312_9.txt #cat command will show the contents in the given path.

Chalk this one up in the win column, this was a superb movie. The acting performances were great and the script was equally great.<br /><br />Helen Hunt was magnificent as the Riverside police officer Gina Pulasky. Gina was a complex character. She was a rookie cop with the Riverside Police Dept. She ended up in an affair with a coworker that she knew had a wife and kids, all the while she took on the dangerous task of going undercover to catch a serial killer. <br /><br />Jeff Fahey (the Ray Liotta look alike) did a bang up job as the confused, often stammering, police officer that had an affair with Gina. He was stoic as an officer, but he was quite the opposite when it came to dealing with his feelings and his extra-marital affair.<br /><br />Steven Weber, most notably from the sitcom "Wings", did a nice job as the quiet, meek, underachieving sociopath. On the surface, he was an innocent loser, but beneath the surface lied a cold hearted killer. Weber really took on the persona of a

In [9]:
import os
os.listdir('aclImdb/train/neg')[:5]

['6925_3.txt', '823_4.txt', '8873_3.txt', '2474_1.txt', '6187_2.txt']

In [10]:
!cat aclImdb/train/neg/6925_3.txt #cat command will show the contents in the given path.

I feel much less generous with this film than others of its ilk. The portrayal of madmen in this century is always done with them being so totally bizarre as to be a different species. Their antics are so outrageous as to be totally fictionalized. Everyone is Napoleon or some other historical figure; or they have a fascination with chickens. They are on the make or beating each other up. It's as if the scriptwriter said, what can I make up for them to do, without an sense of what insanity or even mental illness is. Watch the wonderful human portrayal in "One Flew Over the Cuckoo's Nest" where the illnesses are believable and real. I once worked in a State Mental hospital. I didn't see any of these guys. These are too smart and calculating to make them come to life.

In [0]:
!rm -r aclImdb/train/unsup

*  We have two folders **train, test**. In **train** and **test** , we have another two folders **pos** and **neg** .
*  In **pos** folder, we have all the **positive** reviews.
*  In **neg** folder, we have all the **negative** reviews.

<b> Loading generators to train batch wise</b>

In [12]:
batch_size= 32 # considering batch_size =32

train_directory = tf.keras.preprocessing.text_dataset_from_directory(directory = "aclImdb/train",
                                                                     batch_size=batch_size,
                                                                     validation_split=0.2,
                                                                     subset="training",
                                                                     seed=1337)

validation_directory = tf.keras.preprocessing.text_dataset_from_directory(directory = "aclImdb/train",
                                                                          batch_size=batch_size,
                                                                          validation_split=0.2,
                                                                          subset="validation",
                                                                          seed=1337)

test_directory = tf.keras.preprocessing.text_dataset_from_directory(directory = "aclImdb/test",
                                                                    batch_size=batch_size,
                                                                    seed=1337)

print('Number of batches in train_directory : %d'% tf.data.experimental.cardinality(train_directory))

print('Number of batches in validation_directory : %d'% tf.data.experimental.cardinality(validation_directory))

print('Number of batches in test_directory : %d' % tf.data.experimental.cardinality(test_directory))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in train_directory : 625
Number of batches in validation_directory : 157
Number of batches in test_directory : 782


<b>Previewing few examples</b>

In [13]:
for text_data,label_data in train_directory.take(1):
  for i in range(3):
    print(text_data.numpy()[i])
    print(label_data.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

<b> Preprocessing and preparing the data to train </b>

*  Lowercasing data
*  Removing `<br />` tags
*  Removing punctuations

In [0]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

#custom function

def custom_preprocess(input_data):
  ''' In this function, we will do preprocessing on text'''
  lower_case = tf.strings.lower(input_data)
  cleaned_tags = tf.strings.regex_replace(lower_case,'<br />',' ')
  return tf.strings.regex_replace(cleaned_tags,"[%s]" % re.escape(string.punctuation),'')

# model constraints 

max_features = 20000
embedding_dim = 128
max_sequence_length =500

# instantiating text vectorization layer

vectorize_layer = TextVectorization(max_tokens = max_features, standardize = custom_preprocess, output_mode="int",output_sequence_length = max_sequence_length)

text_data = train_directory.map(lambda x, y: x) # considering only text data , no labels

vectorize_layer.adapt(text_data)

<b> Vectorizing data</b>

Here we are training on GPU. So, below code will help us **asynchronous CPU processing and buffering**.

In [0]:
def vectorize_text(text, label):
  '''In this function, we will expand the dimensions and will vectorize the data'''
  text = tf.expand_dims(text,-1)
  return vectorize_layer(text),label_data

# vectorizing data

train_ds = train_directory.map(vectorize_text)

val_ds =  validation_directory.map(vectorize_text)

test_ds = test_directory.map(vectorize_text)

# Asynchronous prefetching/ buffering on data for better performance on GPU.

train_data = train_ds.cache().prefetch(buffer_size = 10)

val_data = val_d.cache().prefetch(buffer_size = 10)

test_data = test_data.cache().prefetch(buffer_size = 10 )

### **Baseline model**

In [0]:
tf.keras.backend.clear_session()

In [51]:
from tensorflow.keras import layers

# here we  are using functional API
input = tf.keras.Input(shape=(None,),dtype='int64')

embedding_layer = layers.Embedding(max_features+1,embedding_dim)(input)

x = layers.Dropout(0.5)(embedding_layer)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)

x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)

x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)

x = layers.Dropout(0.5)(x)

output = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(input, output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         2560128   
_________________________________________________________________
dropout (Dropout)            (None, None, 128)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         114816    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)              

In [0]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'. Note that we're using max_features+1 here, since there's an
# OOV token that gets added to the vocabulary in vectorize_layer.
x = layers.Embedding(max_features + 1, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [55]:
history=model.fit(train_data,validation_data= val_data,epochs = 3)

Epoch 1/3
625/625 [==============================] - 20s 32ms/step - loss: 0.6863 - accuracy: 0.5616 - val_loss: 0.6853 - val_accuracy: 0.5625
Epoch 2/3
625/625 [==============================] - 19s 31ms/step - loss: 0.6841 - accuracy: 0.5612 - val_loss: 0.6872 - val_accuracy: 0.5625
Epoch 3/3
625/625 [==============================] - 19s 31ms/step - loss: 0.6315 - accuracy: 0.6381 - val_loss: 0.7450 - val_accuracy: 0.4982


In [0]:
history.history